In [1]:
!pip install shap
!pip install xgboost

In [2]:
import pandas as pd
import numpy as np

import boto3
from pyathena import connect
import sagemaker
from sagemaker.feature_store.feature_group import FeatureGroup, FeatureDefinition, FeatureTypeEnum
from sagemaker.session import Session
from pyathena import connect

import time
import shap
import json

import xgboost as xgb
import matplotlib.pyplot as plt
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, roc_auc_score, log_loss

from itertools import combinations
import pickle
import warnings


# S3 and Athena details
bucket_name = "group3-project-bucket"
database_name = "group_project_db"
table_name = "hospital_readmissions"
s3_output = f"s3://{bucket_name}/athena-results/"
region = "us-east-1"
s3_client = boto3.client("s3", region_name=region)

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/sagemaker-user/.config/sagemaker/config.yaml


## Find the latest feature store to generate a new model!

In [3]:
glue_client = boto3.client("glue")

# List databases in AWS Glue
response = glue_client.get_databases()
print("\nAvailable Databases in Glue:")
for db in response["DatabaseList"]:
    print(f"- {db['Name']}")

# List tables in the `sagemaker_featurestore` database (if it exists)
database_name = "sagemaker_featurestore"

try:
    response = glue_client.get_tables(DatabaseName=database_name)
    print(f"\nTables in `{database_name}` database:")
    for table in response["TableList"]:
        print(f"- {table['Name']}")
except glue_client.exceptions.EntityNotFoundException:
    print(f"\nDatabase `{database_name}` not found in Glue.")


Available Databases in Glue:
- default
- group_project_db
- sagemaker_featurestore

Tables in `sagemaker_featurestore` database:
- hospital_readmissions_features_1740354579


Query Athenta Tables for Data Splitting

In [4]:
# Query the feature store in Athena

latest_table = response["TableList"][-1]
query = f"""
SELECT * 
FROM "sagemaker_featurestore"."{latest_table["Name"]}"
"""

# Connect to Athena
connection = connect(
    s3_staging_dir=f"s3://{bucket_name}/athena-results/",
    region_name="us-east-1"
)

# Retrieve all feature data
df = pd.read_sql(query, connection)

/tmp/ipykernel_4696/530253379.py:16: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, connection)


In [5]:
def get_categorical_columns_from_s3(bucket_name):
    """
    Fetches categorical column names and their unique category counts from the S3 stored encoding JSON.
    
    Returns:
        - categorical_columns (list): List of categorical feature names.
        - category_counts (dict): Dictionary where keys are categorical columns and values are the number of unique categories.
    """
    encodings_key = "encodings/encodings.json"
    encodings_file = "encodings.json"

    # Download the encodings file from S3
    s3_client.download_file(bucket_name, encodings_key, encodings_file)

    # Load the encodings JSON
    with open(encodings_file, "r") as f:
        label_encoders = json.load(f)

    # The categorical columns are simply the keys in this JSON
    categorical_columns = list(label_encoders.keys())

    # Compute number of unique categories per categorical column
    category_counts = {col: len(label_encoders[col]) for col in categorical_columns}

    print(f"Identified categorical columns: {categorical_columns}")
    print(f"Category counts per categorical column: {category_counts}")

    return categorical_columns, category_counts

In [6]:
df = df.drop(columns=["event_time", "write_time", "api_invocation_time", "is_deleted"])
display(df.head())


print("Base Features: ", len(df.columns))
print("Data Samples: ", len(df))

categorical_columns, num_cats = get_categorical_columns_from_s3(bucket_name)
print("Categorical columns: ", categorical_columns)
print("Num classes per category: ", num_cats)

age  time_in_hospital  n_lab_procedures  n_procedures  n_medications  \
0    5                 6                62             0             15   
1    5                 5                62             0             20   
2    5                 5                42             0             15   
3    5                 3                39             0             22   
4    5                 6                65             1             14   

   n_outpatient  n_inpatient  n_emergency  medical_specialty  diag_1  diag_2  \
0             0            0            0                  4       6       6   
1             0            0            0                  3       7       0   
2             0            0            0                  2       3       7   
3             0            0            0                  4       6       0   
4             0            1            0                  4       5       6   

   diag_3  glucose_test  a1ctest  change  diabetes_med  readmitted  
0       0             1        1       0             1           0  
1       0             1        0       1             1           0  
2       6             1        1       0             1           0  
3       0             1        1       1             1           1  
4       0             1        1       0             1           0

Base Features:  17
Data Samples:  25000
Identified categorical columns: ['glucose_test', 'A1Ctest', 'age', 'medical_specialty', 'diag_1', 'diag_2', 'diag_3', 'change', 'diabetes_med', 'readmitted']
Category counts per categorical column: {'glucose_test': 3, 'A1Ctest': 3, 'age': 6, 'medical_specialty': 7, 'diag_1': 8, 'diag_2': 8, 'diag_3': 8, 'change': 2, 'diabetes_med': 2, 'readmitted': 2}
Categorical columns:  ['glucose_test', 'A1Ctest', 'age', 'medical_specialty', 'diag_1', 'diag_2', 'diag_3', 'change', 'diabetes_med', 'readmitted']
Num classes per category:  {'glucose_test': 3, 'A1Ctest': 3, 'age': 6, 'medical_specialty': 7, 'diag_1': 8, 'diag_2': 8, 'diag_3': 8, 'change': 2, 'diabetes_med': 2, 'readmitted': 2}


Train an XGB model raw, use SHAP to visual feature importance.

In [7]:
# Split data into train (50%), test (10%), production (40%)
train_df, temp_df = train_test_split(df, test_size=0.5, random_state=42)
test_df, prod_df = train_test_split(temp_df, test_size=0.8, random_state=42)

# Separate features and target variable
target_column = "readmitted"
X_train = train_df.drop(columns=[target_column])
y_train = train_df[target_column]
X_test = test_df.drop(columns=[target_column])
y_test = test_df[target_column]

# Train XGBoost model on production dataset
dmatrix_train = xgb.DMatrix(X_train, label=y_train)
params = {
    "objective": "binary:logistic",
    "eval_metric": "logloss",
    "seed": 42
}
model = xgb.train(params, dmatrix_train, num_boost_round=100)

def eval_model(model, X_test, y_test):
    # Convert test data into DMatrix
    dmatrix_test = xgb.DMatrix(X_test, label=y_test)
    
    # Make predictions
    y_pred_proba = model.predict(dmatrix_test)
    y_pred = (y_pred_proba >= 0.5).astype(int)  # Convert probabilities to binary predictions
    
    # Compute evaluation metrics
    test_log_loss = log_loss(y_test, y_pred_proba)
    test_accuracy = accuracy_score(y_test, y_pred)
    test_auc = roc_auc_score(y_test, y_pred_proba)
    
    # Print evaluation metrics
    print(f"Test Log Loss: {test_log_loss:.4f}")
    print(f"Test Accuracy: {test_accuracy:.4f}")
    print(f"Test AUC: {test_auc:.4f}")


eval_model(model, X_test, y_test)

Test Log Loss: 0.6960
Test Accuracy: 0.5960
Test AUC: 0.6191


Extract Important Features from XGBoost Model.. create interacting features from statistitically most important features.

In [8]:
warnings.filterwarnings("ignore")

def shap_feature_engineering(model, X_train, X_test, bucket_name, top_k=17, shap_threshold=0.01):
    """
    Uses SHAP to find important features and applies interaction rules.
    - Numeric × Numeric → Standard multiplication.
    - Categorical × Categorical → Concatenation.
    - 🚫 **DO NOT COMBINE categorical features with numeric features.**
    - Saves transformation rules for future API inference.
    """
    # ✅ Step 1: Get categorical columns & category counts from S3
    categorical_columns, category_counts = get_categorical_columns_from_s3(bucket_name)

    # ✅ Step 2: Compute SHAP values
    explainer = shap.Explainer(model, X_test)
    shap_values = explainer(X_test)

    # Save SHAP summary plot
    plt.figure()
    shap.summary_plot(shap_values, X_test, show=False)
    plt.savefig("figures/prelim_shap_summary.png")
    plt.close()

    # Compute SHAP feature importance
    feature_importance = pd.DataFrame({
        "feature": X_test.columns,
        "shap_importance": np.abs(shap_values.values).mean(axis=0)
    }).sort_values(by="shap_importance", ascending=False)

    selected_features = feature_importance[feature_importance["shap_importance"] > shap_threshold]["feature"].tolist()
    top_features = feature_importance.head(top_k)["feature"].tolist()
    print("Top Features:", top_features)

    # ✅ Select only the features that passed SHAP importance
    X_train_selected = X_train[selected_features].copy()
    X_test_selected = X_test[selected_features].copy()

    # ✅ Identify categorical and numeric features separately
    top_cat_features = [f for f in top_features if f in categorical_columns]
    top_num_features = [f for f in top_features if f not in categorical_columns]

    # ✅ Step 3: Apply Feature Interactions (Without OneHotEncoding)
    interaction_features = []
    
    for f1, f2 in combinations(top_features[:5], 2):
        if f1 in top_cat_features and f2 in top_cat_features:
            # ✅ Categorical × Categorical → Concatenation
            X_train_selected[f"{f1}_x_{f2}"] = X_train[f1].astype(str) + "_" + X_train[f2].astype(str)
            X_test_selected[f"{f1}_x_{f2}"] = X_test[f1].astype(str) + "_" + X_test[f2].astype(str)
            interaction_features.append((f1, f2))

        elif f1 in top_num_features and f2 in top_num_features:
            # ✅ Numeric × Numeric → Standard multiplication
            X_train_selected[f"{f1}_x_{f2}"] = X_train[f1] * X_train[f2]
            X_test_selected[f"{f1}_x_{f2}"] = X_test[f1] * X_test[f2]
            interaction_features.append((f1, f2))

        else:
            # 🚫 **Categorical × Numeric → SKIPPED**
            print(f"Skipping interaction: {f1} × {f2} (Categorical × Numeric)")

    # ✅ Step 4: Save transformations for API
    interaction_metadata = {
        "interaction_features": interaction_features,
        "categorical_features": categorical_columns
    }

    # Save interaction features
    interaction_file = "figures/interaction_features.json"
    with open(interaction_file, "w") as f:
        json.dump(interaction_metadata, f)

    s3_client.upload_file(interaction_file, bucket_name, "config/interaction_features.json")
    print(f"Interaction features saved to s3://{bucket_name}/config/interaction_features.json")

    return X_train_selected, X_test_selected
    
def apply_interaction_features(X_new, bucket_name):
    """
    Loads transformation metadata from S3 and applies the same interactions for real-time API inference.
    """
    # ✅ Step 1: Load stored interaction transformations
    interaction_key = "config/interaction_features.json"
    interaction_file = "interaction_features.json"
    s3_client.download_file(bucket_name, interaction_key, interaction_file)

    with open(interaction_file, "r") as f:
        interaction_data = json.load(f)

    top_cat_features = interaction_data["categorical_features"]
    interaction_features = interaction_data["interaction_features"]

    # ✅ Step 2: Apply interaction transformations
    for f1, f2 in interaction_features:
        if f1 in top_cat_features and f2 in top_cat_features:
            # ✅ Categorical × Categorical → Concatenation
            X_new[f"{f1}_x_{f2}"] = X_new[f1].astype(str) + "_" + X_new[f2].astype(str)

        elif f1 not in top_cat_features and f2 not in top_cat_features:
            # ✅ Numeric × Numeric → Standard multiplication
            X_new[f"{f1}_x_{f2}"] = X_new[f1] * X_new[f2]

        else:
            # 🚫 **Skipping categorical × numeric interactions**
            print(f"Skipping interaction: {f1} × {f2} (Categorical × Numeric)")

    return X_new

In [9]:
X_train_final, X_test_final = shap_feature_engineering(model, X_train, X_test, bucket_name)
display(X_train_final.head())
display(X_test_final.head())

Identified categorical columns: ['glucose_test', 'A1Ctest', 'age', 'medical_specialty', 'diag_1', 'diag_2', 'diag_3', 'change', 'diabetes_med', 'readmitted']
Category counts per categorical column: {'glucose_test': 3, 'A1Ctest': 3, 'age': 6, 'medical_specialty': 7, 'diag_1': 8, 'diag_2': 8, 'diag_3': 8, 'change': 2, 'diabetes_med': 2, 'readmitted': 2}


 98%|===================| 2449/2500 [00:11<00:00]       

Top Features: ['n_inpatient', 'n_lab_procedures', 'n_medications', 'n_outpatient', 'time_in_hospital', 'diag_1', 'age', 'n_procedures', 'medical_specialty', 'n_emergency', 'diag_2', 'diag_3', 'diabetes_med', 'glucose_test', 'a1ctest', 'change']
Interaction features saved to s3://group3-project-bucket/config/interaction_features.json


n_inpatient  n_lab_procedures  n_medications  n_outpatient  \
12204            0                26             10             0   
2655             0                67             40             0   
9592             0                34              5             0   
18228            0                64             23             0   
18105            0                77             13             0   

       time_in_hospital  diag_1  age  n_procedures  medical_specialty  \
12204                 6       0    2             2                  4   
2655                  8       0    0             6                  6   
9592                  1       6    4             0                  1   
18228                 3       0    2             0                  4   
18105                 4       1    2             2                  3   

       n_emergency  ...  n_inpatient_x_n_lab_procedures  \
12204            0  ...                               0   
2655             0  ...                               0   
9592             0  ...                               0   
18228            1  ...                               0   
18105            0  ...                               0   

       n_inpatient_x_n_medications  n_inpatient_x_n_outpatient  \
12204                            0                           0   
2655                             0                           0   
9592                             0                           0   
18228                            0                           0   
18105                            0                           0   

       n_inpatient_x_time_in_hospital  n_lab_procedures_x_n_medications  \
12204                               0                               260   
2655                                0                              2680   
9592                                0                               170   
18228                               0                              1472   
18105                               0                              1001   

       n_lab_procedures_x_n_outpatient  n_lab_procedures_x_time_in_hospital  \
12204                                0                                  156   
2655                                 0                                  536   
9592                                 0                                   34   
18228                                0                                  192   
18105                                0                                  308   

       n_medications_x_n_outpatient  n_medications_x_time_in_hospital  \
12204                             0                                60   
2655                              0                               320   
9592                              0                                 5   
18228                             0                                69   
18105                             0                                52   

       n_outpatient_x_time_in_hospital  
12204                                0  
2655                                 0  
9592                                 0  
18228                                0  
18105                                0  

[5 rows x 26 columns]

n_inpatient  n_lab_procedures  n_medications  n_outpatient  \
7198            0                64              5             0   
4580            0                53             13             0   
4278            0                43             15             0   
1837            0                69             19             0   
9770            2                62             16             0   

      time_in_hospital  diag_1  age  n_procedures  medical_specialty  \
7198                 2       0    3             0                  4   
4580                 3       6    4             1                  4   
4278                 7       7    1             0                  4   
1837                 6       6    0             1                  4   
9770                 6       3    4             3                  1   

      n_emergency  ...  n_inpatient_x_n_lab_procedures  \
7198            0  ...                               0   
4580            0  ...                               0   
4278            0  ...                               0   
1837            0  ...                               0   
9770            1  ...                             124   

      n_inpatient_x_n_medications  n_inpatient_x_n_outpatient  \
7198                            0                           0   
4580                            0                           0   
4278                            0                           0   
1837                            0                           0   
9770                           32                           0   

      n_inpatient_x_time_in_hospital  n_lab_procedures_x_n_medications  \
7198                               0                               320   
4580                               0                               689   
4278                               0                               645   
1837                               0                              1311   
9770                              12                               992   

      n_lab_procedures_x_n_outpatient  n_lab_procedures_x_time_in_hospital  \
7198                                0                                  128   
4580                                0                                  159   
4278                                0                                  301   
1837                                0                                  414   
9770                                0                                  372   

      n_medications_x_n_outpatient  n_medications_x_time_in_hospital  \
7198                             0                                10   
4580                             0                                39   
4278                             0                               105   
1837                             0                               114   
9770                             0                                96   

      n_outpatient_x_time_in_hospital  
7198                                0  
4580                                0  
4278                                0  
1837                                0  
9770                                0  

[5 rows x 26 columns]

In [11]:
X_prod = prod_df.drop(columns=[target_column])
y_prod = prod_df[target_column]

X_prod_final = apply_interaction_features(X_prod, bucket_name)
print(X_prod_final.columns)

Index(['age', 'time_in_hospital', 'n_lab_procedures', 'n_procedures',
       'n_medications', 'n_outpatient', 'n_inpatient', 'n_emergency',
       'medical_specialty', 'diag_1', 'diag_2', 'diag_3', 'glucose_test',
       'a1ctest', 'change', 'diabetes_med', 'n_inpatient_x_n_lab_procedures',
       'n_inpatient_x_n_medications', 'n_inpatient_x_n_outpatient',
       'n_inpatient_x_time_in_hospital', 'n_lab_procedures_x_n_medications',
       'n_lab_procedures_x_n_outpatient',
       'n_lab_procedures_x_time_in_hospital', 'n_medications_x_n_outpatient',
       'n_medications_x_time_in_hospital', 'n_outpatient_x_time_in_hospital'],
      dtype='object')


Bayesian Optimization Procedure to find best XGB model

In [12]:
!pip install optuna

In [13]:
import optuna
from sklearn.metrics import roc_auc_score

def xgb_objective(trial):
    params = {
        'objective': 'binary:logistic',
        'eval_metric': 'logloss',
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.3),
        'max_depth': trial.suggest_int('max_depth', 3, 10),
        'min_child_weight': trial.suggest_float('min_child_weight', 1, 10),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.3, 1.0),
        'subsample': trial.suggest_float('subsample', 0.5, 1.0),
        'seed': 42
    }
    
    dtrain = xgb.DMatrix(X_train_final, label=y_train)
    dtest = xgb.DMatrix(X_test_final, label=y_test)
    model = xgb.train(params, dtrain, num_boost_round=100)
    preds = model.predict(dtest)
    
    return roc_auc_score(y_test, preds)

study = optuna.create_study(direction="maximize")
study.optimize(xgb_objective, n_trials=30)

best_params = study.best_params
best_params['max_depth'] = int(best_params['max_depth'])

# Train the best model
dmatrix_train = xgb.DMatrix(X_train_final, label=y_train)
model = xgb.train(best_params, dmatrix_train, num_boost_round=500)

[I 2025-02-24 01:50:09,305] A new study created in memory with name: no-name-b5c9a398-3712-44e5-8a55-ba28e7ffc348
[I 2025-02-24 01:50:09,739] Trial 0 finished with value: 0.6296632751937984 and parameters: {'learning_rate': 0.14614232706897776, 'max_depth': 7, 'min_child_weight': 2.0925888083130193, 'colsample_bytree': 0.47844572384618583, 'subsample': 0.6540120786291136}. Best is trial 0 with value: 0.6296632751937984.
[I 2025-02-24 01:50:10,059] Trial 1 finished with value: 0.6627022579057462 and parameters: {'learning_rate': 0.11212382451469148, 'max_depth': 4, 'min_child_weight': 8.530513367384355, 'colsample_bytree': 0.7096867096158481, 'subsample': 0.6222573455503895}. Best is trial 1 with value: 0.6627022579057462.
[I 2025-02-24 01:50:10,510] Trial 2 finished with value: 0.6171826934908331 and parameters: {'learning_rate': 0.26099513235914534, 'max_depth': 7, 'min_child_weight': 1.2861241887234272, 'colsample_bytree': 0.7392243445971078, 'subsample': 0.7815268724622162}. Best is

In [14]:
# Visualize Optuna Trials
fig = optuna.visualization.matplotlib.plot_optimization_history(study)
plt.savefig("figures/optuna_optimization_history.png")
plt.close()

fig = optuna.visualization.matplotlib.plot_param_importances(study)
plt.savefig("figures/optuna_param_importance.png")
plt.close()

In [15]:
explainer = shap.Explainer(model, X_test_final)
shap_values = explainer(X_test_final)

# Save SHAP summary plot
plt.figure()
shap.summary_plot(shap_values, X_test_final, show=False)
plt.savefig("figures/final_shap_summary.png")
plt.close()

# Save SHAP dependence plot for the first feature
plt.figure()
shap.dependence_plot(0, shap_values.values, X_test_final, show=False)
plt.savefig("figures/final_shap_dependence_0.png")
plt.close()

 97%|=================== | 2437/2500 [00:15<00:00]       

<Figure size 640x480 with 0 Axes>

In [16]:
eval_model(model, X_test_final, y_test)

Test Log Loss: 0.6565
Test Accuracy: 0.6188
Test AUC: 0.6547


In [26]:
# Save the model
model.save_model("models/tuned_xgboost_model.model")
print("Model saved to models/tuned_xgboost_model.model")

Model saved to models/tuned_xgboost_model.model


/opt/conda/lib/python3.11/site-packages/xgboost/core.py:158: UserWarning: [23:56:32] WARNING: /workspace/src/c_api/c_api.cc:1374: Saving model in the UBJSON format as default.  You can use file extension: `json`, `ubj` or `deprecated` to choose between formats.
  warnings.warn(smsg, UserWarning)
